<a href="https://colab.research.google.com/github/FlyFish0203/flyfish/blob/main/ex04_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

問題定義
將 性別 欄位 Label Encoding

將客戶分成 3 群，並解讀各群特質

計算 k = 2 ~ 15 的 Calinski-Harbasz Score，找出最佳 k 值

In [1]:
!wget -O car_models.csv https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-customer.csv

--2025-05-27 08:21:18--  https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-customer.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2839 (2.8K) [text/plain]
Saving to: ‘car_models.csv’

car_models.csv      100%[===================>]   2.77K  --.-KB/s    in 0s      

2025-05-27 08:21:18 (33.8 MB/s) - ‘car_models.csv’ saved [2839/2839]



In [2]:
import pandas as pd
url = 'https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-customer.csv'
df = pd.read_csv(url)
df.head()

,性別,年齡,收入（千）,消費指數（1~100）
0,女,74,38,81
1,女,51,71,91
2,女,30,65,10
3,女,88,49,17
4,女,55,48,70


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   性別           200 non-null    object
 1   年齡           200 non-null    int64 
 2   收入（千）        200 non-null    int64 
 3   消費指數（1~100）  200 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 6.4+ KB


In [4]:
df_cor = df.drop(columns='性別').corr()
df_cor

,年齡,收入（千）,消費指數（1~100）
年齡,1.000000,0.031519,-0.127454
收入（千）,0.031519,1.000000,0.031476
消費指數（1~100）,-0.127454,0.031476,1.000000


In [5]:
df['性別'].unique()

array(['女', '男'], dtype=object)

In [6]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(categories=[['女','男']])
df[['性別']] = encoder.fit_transform(df[['性別']])
df

,性別,年齡,收入（千）,消費指數（1~100）
0,0.0,74,38,81
1,0.0,51,71,91
2,0.0,30,65,10
3,0.0,88,49,17
4,0.0,55,48,70
...,...,...,...,...
195,1.0,86,84,82
196,1.0,59,52,30
197,0.0,63,29,61
198,1.0,67,80,9


In [7]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3) #建立一個 KMeans 模型，指定要將資料分成 3 群
kmeans.fit(df) #對資料集 df 執行 K-Means 分群的訓練過程

KMeans(n_clusters=3)

In [8]:
# 為每筆資料加上分群結果
df['cluster'] = kmeans.labels_
df

,性別,年齡,收入（千）,消費指數（1~100）,cluster
0,0.0,74,38,81,2
1,0.0,51,71,91,1
2,0.0,30,65,10,0
3,0.0,88,49,17,0
4,0.0,55,48,70,2
...,...,...,...,...,...
195,1.0,86,84,82,1
196,1.0,59,52,30,0
197,0.0,63,29,61,2
198,1.0,67,80,9,0


In [9]:
df.groupby('cluster').mean()

,性別,年齡,收入（千）,消費指數（1~100）
cluster,,,,
0,0.592105,57.842105,60.381579,19.934211
1,0.607843,46.254902,82.764706,75.176471
2,0.520548,52.150685,40.479452,70.726027


In [10]:
# 評估分群效果
# 使用 Calinski-Harabasz Index（CH 指標） 來評估 KMeans 所分出來的群的「群內緊密度」和「群間分離度」
from sklearn.metrics import calinski_harabasz_score
score = calinski_harabasz_score(df.drop(columns='cluster'), kmeans.labels_)
score


np.float64(96.58085896888946)

In [11]:
df_nocluster = df.drop(columns='cluster')
for i in range(2,16):
  kmeans = KMeans(n_clusters=i)
  kmeans.fit(df_nocluster)
  score = calinski_harabasz_score(df_nocluster, kmeans.labels_)
  print(f'k={i} score={score}')


k=2 score=111.68453199285265
k=3 score=95.49707423522689
k=4 score=91.21650751454469
k=5 score=90.1077377926896
k=6 score=94.32914544951379
k=7 score=91.34844827103919
k=8 score=100.67205007595922
k=9 score=80.61101665268451
k=10 score=81.78247443408802
k=11 score=86.6318045326734
k=12 score=83.41002641293716
k=13 score=86.1324193395287
k=14 score=83.52008681430749
k=15 score=80.47426214423372
